In [1]:
from ultralytics import YOLO
import cv2
import torch
import numpy as np
import os

In [2]:
def segmentation_to_bbox(masks):
    segmentation = np.where(masks)

    # Bounding Box
    bbox = 0, 0, 0, 0
    if len(segmentation) != 0 and len(segmentation[1]) != 0 and len(segmentation[0]) != 0:
        x_min = int(np.min(segmentation[1]))
        x_max = int(np.max(segmentation[1]))
        y_min = int(np.min(segmentation[0]))
        y_max = int(np.max(segmentation[0]))

        bbox = x_min, x_max, y_min, y_max

    return bbox


def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

In [4]:
model = YOLO('27-june.pt')

In [42]:
filepath = "Data Test for BDC 2023 - Penyisihan/" + "DataTest9.png"
# filepath = "output_bright.jpg"

In [44]:
results = model(filepath, imgsz=640)
img = cv2.imread(filepath)

for result in results:
    for mask in result.masks:
        dim = mask.data.shape
        img = cv2.resize(img, (dim[2], dim[1]))

        m = torch.squeeze(mask.data)
        composite = torch.stack((m, m, m), 2)
        tmp = img * composite.cpu().numpy().astype(np.uint8)
        crop = tmp
        cv2.imshow("original", img)
        cv2.imshow("result", tmp)
        cv2.waitKey(0)
        break
    break
cv2.destroyAllWindows()


image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest9.png: 256x640 1 License Plate, 16.6ms
Speed: 1.0ms preprocess, 16.6ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)


In [36]:
masks = results[0].masks[0].data[0].cpu().numpy()

In [37]:
cropped = crop.copy()
gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (3, 3), 0)
canny = cv2.Canny(gray, 20, 50)
kernel = np.ones((3, 3), np.uint8)
dilation = cv2.dilate(canny, kernel, iterations=2)
erosion = cv2.erode(dilation, kernel, iterations=1)


cv2.imshow("erosion", erosion)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
_, thresh = cv2.threshold(erosion, 45, 255, cv2.THRESH_BINARY)
cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)

In [39]:
# find biggest contour
c = max(cnts, key=cv2.contourArea)

peri = cv2.arcLength(c, True)
approx = cv2.approxPolyDP(c, 0.02 * peri, True)

In [40]:
circle = cropped.copy()
for i in range(len(approx)):
    cv2.circle(circle, (approx[i][0][0], approx[i][0][1]), 3, (0, 255, 0), -1)

cv2.imshow("circle", circle)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [41]:
perspective = []
rect = approx
for i in range(len(approx)):
    point_before = approx[i % len(approx)][0]
    perspective.append((point_before[0], point_before[1]))
    point_after = approx[(i + 1) % len(approx)][0]
    cv2.line(cropped, point_before, point_after, (0, 255, 0), 2)

perspective = order_points(np.array(perspective))

cv2.imshow("original", img)
cv2.imshow("cropped", cropped)

height, width, channels = cropped.shape

# pts1 = np.float32([[0, 0], [width - 150, 0], [width, height], [0, height]])
pts1 = np.float32(perspective)
pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
matrix = cv2.getPerspectiveTransform(pts1, pts2)
transformed = cv2.warpPerspective(cropped, matrix, (width, height))

cv2.imshow("transformed", transformed)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Pipelining

In [11]:
unprocessed_images = []

def predict(filepath, model):
    return model.predict(filepath, imgsz=640)

def pipeline_stage1(filepath, target_dir, model):
    results = predict(filepath, model)
    img = cv2.imread(filepath)

    for result in results:
        if result.masks is None:
            result_path = os.getcwd() + "\\" + target_dir + "\\" + os.path.basename(filepath)
            print("WARNING: No mask found for image " + os.path.basename(filepath) + ". Saving original image.")
            unprocessed_images.append(os.path.basename(filepath))
            cv2.imwrite(result_path, img)
            return

        for mask in result.masks:
            dim = mask.data.shape
            img = cv2.resize(img, (dim[2], dim[1]))

            m = torch.squeeze(mask.data)
            composite = torch.stack((m, m, m), 2)
            tmp = img * composite.cpu().numpy().astype(np.uint8)
            crop = tmp
            break
        break


    cropped = crop.copy()
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    canny = cv2.Canny(gray, 20, 50)
    kernel = np.ones((3, 3), np.uint8)
    dilation = cv2.dilate(canny, kernel, iterations=2)
    erosion = cv2.erode(dilation, kernel, iterations=1)

    _, thresh = cv2.threshold(erosion, 45, 255, cv2.THRESH_BINARY)
    cnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)

    c = max(cnts, key=cv2.contourArea)

    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)

    perspective = []
    for i in range(len(approx)):
        point_before = approx[i % len(approx)][0]
        perspective.append((point_before[0], point_before[1]))
        point_after = approx[(i + 1) % len(approx)][0]
        # cv2.line(cropped, point_before, point_after, (0, 255, 0), 2)

    perspective = order_points(np.array(perspective))

    height, width, channels = cropped.shape

    # pts1 = np.float32([[0, 0], [width - 150, 0], [width, height], [0, height]])
    pts1 = np.float32(perspective)
    pts2 = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    transformed = cv2.warpPerspective(cropped, matrix, (width, height))

    os.makedirs(target_dir, exist_ok=True)

    result_path = os.getcwd() + "\\" + target_dir + "\\" + os.path.basename(filepath)
    cv2.imwrite(result_path, transformed)
    print("Saved " + result_path)

pipeline_stage1("output_bright.jpg", "stage_1", model)


image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\output_bright.jpg: 192x640 1 License Plate, 134.3ms
Speed: 3.0ms preprocess, 134.3ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\stage_1\output_bright.jpg


In [13]:
# scan raw_data/images directory
# for each image, run pipeline_stage1
# save result to raw_data/stage_1 directory
file_list = os.listdir("Data Test for BDC 2023 - Penyisihan")
for file in file_list:
    pipeline_stage1("Data Test for BDC 2023 - Penyisihan/" + file, "raw_data/test", model)


image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest1.png: 224x640 1 License Plate, 155.6ms
Speed: 4.0ms preprocess, 155.6ms inference, 7.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest10.png: 256x640 1 License Plate, 102.2ms
Speed: 3.0ms preprocess, 102.2ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest100.png: 192x640 1 License Plate, 15.7ms
Speed: 1.5ms preprocess, 15.7ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest11.png: 224x640 1 License Plate, 17.6ms


Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest1.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest10.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest100.png


Speed: 1.0ms preprocess, 17.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest12.png: 224x640 1 License Plate, 16.7ms
Speed: 1.0ms preprocess, 16.7ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest13.png: 224x640 1 License Plate, 17.1ms
Speed: 0.5ms preprocess, 17.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest14.png: 224x640 1 License Plate, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest15.png: 352x640 1 License Plate, 20.5ms
Speed: 2.0ms preprocess, 20.5ms inference, 3.0ms pos

Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest11.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest12.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest13.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest14.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest15.png


Speed: 1.0ms preprocess, 18.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest17.png: 224x640 1 License Plate, 19.7ms
Speed: 1.0ms preprocess, 19.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest18.png: 224x640 1 License Plate, 19.0ms
Speed: 1.0ms preprocess, 19.0ms inference, 6.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest19.png: 224x640 1 License Plate, 14.8ms
Speed: 1.0ms preprocess, 14.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest2.png: 224x640 1 License Plate, 15.6ms
Speed: 1.0ms preprocess, 15.6ms inference, 3.0ms post

...

Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest85.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest86.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest87.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest88.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest89.png


Speed: 1.0ms preprocess, 16.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest90.png: 224x640 1 License Plate, 17.0ms
Speed: 1.0ms preprocess, 17.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest91.png: 256x640 1 License Plate, 18.0ms
Speed: 1.0ms preprocess, 18.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest92.png: 288x640 1 License Plate, 18.6ms
Speed: 1.5ms preprocess, 18.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest93.png: 288x640 1 License Plate, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 3.0ms pos

Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest9.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest90.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest91.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest92.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest93.png


Speed: 2.0ms preprocess, 16.1ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest95.png: 160x640 1 License Plate, 22.5ms
Speed: 1.0ms preprocess, 22.5ms inference, 7.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest96.png: 256x640 2 License Plates, 17.6ms
Speed: 1.0ms preprocess, 17.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest97.png: 192x640 1 License Plate, 15.0ms
Speed: 0.0ms preprocess, 15.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\vitos\Files\Projects\bdc\penyisihan\Data Test for BDC 2023 - Penyisihan\DataTest98.png: 192x640 1 License Plate, 14.0ms
Speed: 1.0ms preprocess, 14.0ms inference, 3.0ms po

Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest94.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest95.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest96.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest97.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest98.png
Saved c:\Users\vitos\Files\Projects\bdc\penyisihan\raw_data/test\DataTest99.png
